# Learning CFR: 

### Regret matching: 

in a game of Rock, Paper, Scissor, you play rock and your opponent plays paper and you lose. You regret not having played paper or scissor. You want to use this to inform your next round of play. 

#### Definitions:

Action ($S_i$): the choices available for players eg R, P, S

Each possible combination of simultaneous actions -> action profile (eg: I play Rock and he plays Paper, 'R,P' is now an action profile. $A = S_1 x S_2 x ... x S_n$

Utility: reward and punishment of an action provided by a function $u_i()$ eg if we play for 1 dollar then utilities are +1, 0, -1

Strategy: the probability of choosing a single action $\sigma(s)$

Expected utility (for a player): player probability X utility summed for each action profile for both players

$u_i (\sigma_i, \sigma_-i) = \sum \sum \sigma_i(s)  \sigma_-i(s') u(s,s') $

#### The Regret matching algorithm:

Regret (for an action we have not chosen): utility of action we didn't choose - utility of action we did choose

eg Regret of not playing paper =  $u(paper, paper) - u(rock, paper)$

How do we inform future plays based on that? 

We choose the action that we regret most not choosing. But, to avoid being predectbile, we select actions at random from a distribution proportional to positive regrets. 

Normalized regrets: to get proabulities -> positive regrets / their sum 

eg in RPS example: R,P,S -> $0, 1, 2 -> 0/3, 1/3, 2/3 -> (0, 1/3, 2/3)$

We add previous regrets each step so we get cumulative regrets.

#### The algorithm is as follows: 

```
For each player, initialize all cumulative regrets to 0. 

For some number of iterations: 
- Compute a regret-matching strategy profile. (If all regrets for a player are non-positive, use a uniform random strategy.) 
- Add the strategy profile to the strategy profile sum. 
- Select each player action profile according the strategy profile. 
- Compute player regrets. 
- Add player regrets to player cumulative regrets.
- Return the average strategy profile, i.e. the strategy profile sum divided by the number of iterations.
```

#### Concepts inventory:

Strategy profile: we need a probability distribution for each player (a function that computes it). 

Action: a function that selects next action at random from the probability distribution (strategy profile). 

Utility: we need a function to compute utilities for each player based on their selected actions eg $u(R, S)$

Regrets: we need to compute regrets based on utilities (see above). 

In [1]:
import random

In [11]:
random.sample(['R', 'P', 'S'], 1)

['P']

In [123]:
# one player - opponent - will be static [0.3, 0.3, 0.4] -> this is his Strategy Profile
# we need to compute it for the other player
# R = 0, P = 1, S = 2 -> we represent them as indices

actions = [0, 1, 2]

In [131]:
def getRegrets(a):
    '''
    Takes as input a -> action_profile [0,0] returns [0, 1, -1] for (R, P, S).
    We do calculation for one player since opponent is fixed.
    '''
    regrets = [0, 0, 0]
    acts = [0, 1, 2]
    player_utilities = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
    
    regrets[a[0]] = 0 # we have no regrets for the action we choose
    acts.remove(a[0])    
    
    for i in actions:
        # regret = utility(not chosen) - utility(chosen)
        regrets[i] = player_utilities[i, a[1]] - player_utilities[a[0], a[1]]
    return regrets

In [132]:
r = getRegrets([0,1])
r

[0, 1, 2]

In [104]:
r / sum(r)

array([0.        , 0.33333333, 0.66666667])

In [19]:
import numpy as np

In [75]:
player_utilities = np.array([[0, -1, 1], [1, 0, -1], [-1, 1, 0]])
a = [0, 1]
player_utilities[a[0], a[1]]

-1

In [149]:
def getStrategy(regrets):
    '''
    Return a regret-matching strategy profile of the form [0.3, 0.4, 0.4]
    If all regrets for a player are non-positive, use a uniform random strategy.
    '''
    if sum(regrets) <= 0: return 1.0 / np.array([3, 3, 3])
    else: 
        normalized_regrets = regrets / sum(regrets)
        
    return normalized_regrets

In [150]:
def select_action_profile(playerSt, opponentSt):
    player = np.random.choice(a = actions, p = playerSt)
    opponent = np.random.choice(a = actions, p = opponentSt)
    return [player, opponent]

In [151]:
select_action_profile(np.array([0.3, 0.3, 0.4]), np.array([0.3, 0.3, 0.4]))

[2, 1]

In [152]:
playerRegrets

array([0., 0., 0.])

In [169]:
#opponentStrategy = np.array([0.3, 0.3, 0.4])

opponentStrategy = np.array([0.0, 0.0, 1.0])

playerStrategy = np.zeros(3)
playerRegrets = np.zeros(3)
cumulativeRegrets = np.zeros(3)
strategySum = np.zeros(3)

iters = 1000

for i in range(iters):
    playerStrategy = getStrategy(playerRegrets)
    strategySum += playerStrategy
    action_profile = select_action_profile(playerStrategy, opponentStrategy)
    playerRegrets = getRegrets(action_profile)
    cumulativeRegrets += playerRegrets

print(strategySum / iters)

[0.41566667 0.251      0.33333333]


In [168]:
sum((strategySum / iters))

0.9999999999999997